# The `open` event

The `open` audit event is raised when a file is opened by the `open` builtin
and some other mechanisms.

This notebook shows one other function that also raises the `open` event, but
there are others not shown here.

In [1]:
import os
from pathlib import Path
import platform

import subaudit

In [2]:
# Support both VS Code and JupyterLab, which give notebooks different CWDs.
if not Path('LICENSE').exists():
    grandparent = Path().absolute().parent.parent
    assert (grandparent / 'LICENSE').exists()
    os.chdir(grandparent)

# We'll open this other file, too, in another example.
assert Path('README.md').exists()

## From the `open` builtin

In [3]:
with subaudit.extracting('open', lambda *args: (*args,)) as extracts:
    with open('LICENSE', encoding='utf-8'):
        pass

extracts

[('LICENSE', 'r', 524288)]

On CPython, the above output tends to look like:

```python
[('README.md', 'r', 524288)]
```

The value of flags (the third argument) can to vary across platforms.

On PyPy, it tends to look like:

```python
[('LICENSE', 'r', 1)]
```

## From `pathlib.Path`’s `read_*` and `write_*`

Using a `pathlib.Path` object to open a file, by calling its `read_bytes`,
`read_text`, `write_bytes`, or `write_text` methods, likewise raises the `open`
event.

In Python 3.10 and later, a call to one of those methods raises the `open`
event only once.

Prior to 3.10, a call to one of those methods raises the `open` event *twice*:
in effect, the method raises it on behalf of itself as well. This can be
distinguished because it passes `None` for the `mode` argument. In addition, on
PyPy, this call passes a `Path` object rather than `str`, for the `path`
argument. This additional event is raised *after* the regular one.

In [4]:
with subaudit.extracting('open', lambda *args: (*args,)) as extracts:
    Path('README.md').read_text(encoding='utf-8')

extracts

[('README.md', 'r', 524288)]

On Python 3.10 and higher, the above output tends to look like:

```python
[('README.md', 'r', 524288)]
```

On CPython 3.9 and earlier, it tends to look like:

```python
[('README.md', 'r', 524288), ('README.md', None, 524288)]
```

On PyPy, when providing 3.9 or earlier, it tends to look like:

```python
[('README.md', 'r', 1), (PosixPath('README.md'), None, 524288)]
```

Note: The testing to confirm the PyPy behavior was done with PyPy 3.8 only.

In [5]:
platform.python_implementation()

'CPython'

In [6]:
platform.python_version()

'3.11.3'